### Elasticity tensor

The elasticity tensor is defined as follows:

\begin{equation*}
    \sigma _{ij} = \mathcal{C _{ijkl}} \, e_{kl}
\end{equation*}

Where $\sigma _{ij}$ is the stress tensor and $e _{kl}$ the deformation tensor, or rather, the Green-Lagrange strain tensor approximated to its symmetric part.
Under a basis transformation, the elasticity tensor follows the law:

\begin{equation*}
    \mathcal{C' _{ijkl}} = M _{im} \, M _{jn} \, M _{kp} \, M _{lq} \, \mathcal{C_{mnpq}}
\end{equation*}

Where $M$ is the basis transition matrix

In [1]:
# Importing SymPy
import sympy as smp

In [2]:
# Transforming the tensor in the new basis
def transformer(M, type = 'P', *args):

    # Defining the elasticity tensor C_{ijkl}
    C_symbols = smp.symbols('C_1:4_1:4_1:4_1:4', real = True)
    C = smp.MutableDenseNDimArray(C_symbols, (3, 3, 3, 3))

    # Main symmetries of C
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    C[j, i, k, l] = C[i, j, k, l]  # Swap i <-> j
                    C[i, j, l, k] = C[i, j, k, l]  # Swap k <-> l
                    C[k, l, i, j] = C[i, j, k, l]  # Swap ij <-> kl
        
    C_prime = smp.MutableDenseNDimArray.zeros(3, 3, 3, 3)
    
    # The transformation of the tensor C -> C' is done using the matrix M
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    C_prime[i, j, k, l] = sum(
                        M[i, m] * M[j, n] * M[k, o] * M[l, p] * C[m, n, o, p]
                        for m in range(3) for n in range(3) for o in range(3) for p in range(3)
                        )
    
    matrix_6x6 = smp.Matrix.zeros(6, 6)

    # Mapping function for 6x6 matrix
    def map_indices(i, j):
        mapping = {
            (0, 0): 0, (1, 1): 1, (2, 2): 2,  # Diagonal: xx, yy, zz
            (0, 1): 3, (1, 0): 3,   # xy, yx -> 3
            (0, 2): 4, (2, 0): 4,  # xz, zx -> 4
            (1, 2): 5, (2, 1): 5,  # yz, zy -> 5
        }
        return mapping.get((i, j))

    # Populating the 6x6 matrix with the mapped components of C_prime
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    p = map_indices(i, j)
                    q = map_indices(k, l)

                    if p is not None and q is not None:
                        matrix_6x6[p, q] = C_prime[i, j, k, l]
                    
                    # Only for infinitesimal rotation matrices
                    if type == 'R':
                        subs = {
                            args[0] ** 2: 0,
                            args[0] ** 3: 0,
                            args[0] ** 4: 0
                            }
                        
                        matrix_6x6 = matrix_6x6.subs(subs).applyfunc(lambda Q: smp.collect(Q, args[0]))
                        
    return matrix_6x6

Let's see some key basis changes. We start studying the elasticity tensor under infinitesimal rotations

In [3]:
# Code for infinitesimal rotations
dtheta = smp.symbols('d\\theta', real = True, constant = True)

lx, ly, lz = smp.symbols("l_x, l_y, l_z", real = True, constant = True)
l = smp.Matrix([lx,
                ly,
                lz])
l = l / l.norm()

# Rodrigues formula
K = smp.Matrix([[0, - l[2], l[1]],
                [l[2], 0, - l[0]],
                [- l[1], l[0], 0]])

R = smp.eye(3) + (smp.sin(dtheta) * K) + ((1 - smp.cos(dtheta)) * (K * K))
R = R.subs([(smp.cos(dtheta), 1),
            (smp.sin(dtheta), dtheta)]) # from theta to dtheta

Rx = R.subs([(lx, 1),
             (ly, 0),
             (lz, 0)])

Ry = R.subs([(lx, 0),
             (ly, 1),
             (lz, 0)])

Rz = R.subs([(lx, 0),
             (ly, 0),
             (lz, 1)])

R111 = R.subs([(lx, 1),
               (ly, 1),
               (lz, 1)])

In [4]:
# Mapping the original C
transformer(smp.eye(3))

Matrix([
[C_1_1_1_1, C_1_1_2_2, C_1_1_3_3, C_1_1_1_2, C_1_1_1_3, C_1_1_2_3],
[C_1_1_2_2, C_2_2_2_2, C_2_2_3_3, C_1_2_2_2, C_1_3_2_2, C_2_2_2_3],
[C_1_1_3_3, C_2_2_3_3, C_3_3_3_3, C_1_2_3_3, C_1_3_3_3, C_2_3_3_3],
[C_1_1_1_2, C_1_2_2_2, C_1_2_3_3, C_1_2_1_2, C_1_2_1_3, C_1_2_2_3],
[C_1_1_1_3, C_1_3_2_2, C_1_3_3_3, C_1_2_1_3, C_1_3_1_3, C_1_3_2_3],
[C_1_1_2_3, C_2_2_2_3, C_2_3_3_3, C_1_2_2_3, C_1_3_2_3, C_2_3_2_3]])

In [5]:
# General infinitesimal rotation
transformer(R, 'R', dtheta)

Matrix([
[                                                                                                                                                                                            C_1_1_1_1 + d\theta*(-4*C_1_1_1_2*l_z/sqrt(l_x**2 + l_y**2 + l_z**2) + 4*C_1_1_1_3*l_y/sqrt(l_x**2 + l_y**2 + l_z**2)),                                                                                            C_1_1_2_2 + d\theta*(2*C_1_1_1_2*l_z/sqrt(l_x**2 + l_y**2 + l_z**2) - 2*C_1_1_2_3*l_x/sqrt(l_x**2 + l_y**2 + l_z**2) - 2*C_1_2_2_2*l_z/sqrt(l_x**2 + l_y**2 + l_z**2) + 2*C_1_3_2_2*l_y/sqrt(l_x**2 + l_y**2 + l_z**2)),                                                                                           C_1_1_3_3 + d\theta*(-2*C_1_1_1_3*l_y/sqrt(l_x**2 + l_y**2 + l_z**2) + 2*C_1_1_2_3*l_x/sqrt(l_x**2 + l_y**2 + l_z**2) - 2*C_1_2_3_3*l_z/sqrt(l_x**2 + l_y**2 + l_z**2) + 2*C_1_3_3_3*l_y/sqrt(l_x**2 + l_y**2 + l_z**2)),                                                                   

In [6]:
transformer(Rx, 'R', dtheta)

Matrix([
[                                  C_1_1_1_1,                           C_1_1_2_2 - 2*C_1_1_2_3*d\theta,                           2*C_1_1_2_3*d\theta + C_1_1_3_3,                           C_1_1_1_2 - C_1_1_1_3*d\theta,                           C_1_1_1_2*d\theta + C_1_1_1_3,               C_1_1_2_3 + d\theta*(C_1_1_2_2 - C_1_1_3_3)],
[            C_1_1_2_2 - 2*C_1_1_2_3*d\theta,                           C_2_2_2_2 - 4*C_2_2_2_3*d\theta,           C_2_2_3_3 + d\theta*(2*C_2_2_2_3 - 2*C_2_3_3_3),          C_1_2_2_2 + d\theta*(-2*C_1_2_2_3 - C_1_3_2_2),           C_1_3_2_2 + d\theta*(C_1_2_2_2 - 2*C_1_3_2_3), C_2_2_2_3 + d\theta*(C_2_2_2_2 - C_2_2_3_3 - 2*C_2_3_2_3)],
[            2*C_1_1_2_3*d\theta + C_1_1_3_3,           C_2_2_3_3 + d\theta*(2*C_2_2_2_3 - 2*C_2_3_3_3),                           4*C_2_3_3_3*d\theta + C_3_3_3_3,           C_1_2_3_3 + d\theta*(2*C_1_2_2_3 - C_1_3_3_3),           C_1_3_3_3 + d\theta*(C_1_2_3_3 + 2*C_1_3_2_3), C_2_3_3_3 + d\theta*(C_2_2_3_3 + 2*C_

In [7]:
transformer(Ry, 'R', dtheta)

Matrix([
[                           C_1_1_1_1 + 4*C_1_1_1_3*d\theta,              C_1_1_2_2 + 2*C_1_3_2_2*d\theta,           C_1_1_3_3 + d\theta*(-2*C_1_1_1_3 + 2*C_1_3_3_3),            C_1_1_1_2 + d\theta*(C_1_1_2_3 + 2*C_1_2_1_3), C_1_1_1_3 + d\theta*(-C_1_1_1_1 + C_1_1_3_3 + 2*C_1_3_1_3),           C_1_1_2_3 + d\theta*(-C_1_1_1_2 + 2*C_1_3_2_3)],
[                           C_1_1_2_2 + 2*C_1_3_2_2*d\theta,                                    C_2_2_2_2,                           -2*C_1_3_2_2*d\theta + C_2_2_3_3,                            C_1_2_2_2 + C_2_2_2_3*d\theta,               C_1_3_2_2 + d\theta*(-C_1_1_2_2 + C_2_2_3_3),                           -C_1_2_2_2*d\theta + C_2_2_2_3],
[          C_1_1_3_3 + d\theta*(-2*C_1_1_1_3 + 2*C_1_3_3_3),             -2*C_1_3_2_2*d\theta + C_2_2_3_3,                           -4*C_1_3_3_3*d\theta + C_3_3_3_3,           C_1_2_3_3 + d\theta*(-2*C_1_2_1_3 + C_2_3_3_3), C_1_3_3_3 + d\theta*(-C_1_1_3_3 - 2*C_1_3_1_3 + C_3_3_3_3),           C_2_3_3_

In [8]:
transformer(Rz, 'R', dtheta)

Matrix([
[                          C_1_1_1_1 - 4*C_1_1_1_2*d\theta,           C_1_1_2_2 + d\theta*(2*C_1_1_1_2 - 2*C_1_2_2_2),             C_1_1_3_3 - 2*C_1_2_3_3*d\theta, C_1_1_1_2 + d\theta*(C_1_1_1_1 - C_1_1_2_2 - 2*C_1_2_1_2),          C_1_1_1_3 + d\theta*(-C_1_1_2_3 - 2*C_1_2_1_3),           C_1_1_2_3 + d\theta*(C_1_1_1_3 - 2*C_1_2_2_3)],
[          C_1_1_2_2 + d\theta*(2*C_1_1_1_2 - 2*C_1_2_2_2),                           4*C_1_2_2_2*d\theta + C_2_2_2_2,             2*C_1_2_3_3*d\theta + C_2_2_3_3, C_1_2_2_2 + d\theta*(C_1_1_2_2 + 2*C_1_2_1_2 - C_2_2_2_2),           C_1_3_2_2 + d\theta*(2*C_1_2_1_3 - C_2_2_2_3),           C_2_2_2_3 + d\theta*(2*C_1_2_2_3 + C_1_3_2_2)],
[                          C_1_1_3_3 - 2*C_1_2_3_3*d\theta,                           2*C_1_2_3_3*d\theta + C_2_2_3_3,                                   C_3_3_3_3,               C_1_2_3_3 + d\theta*(C_1_1_3_3 - C_2_2_3_3),                           C_1_3_3_3 - C_2_3_3_3*d\theta,                           C_1_3_3_3

In [9]:
transformer(R111, 'R', dtheta)

Matrix([
[                                                                                        C_1_1_1_1 + d\theta*(-4*sqrt(3)*C_1_1_1_2/3 + 4*sqrt(3)*C_1_1_1_3/3),                                          C_1_1_2_2 + d\theta*(2*sqrt(3)*C_1_1_1_2/3 - 2*sqrt(3)*C_1_1_2_3/3 - 2*sqrt(3)*C_1_2_2_2/3 + 2*sqrt(3)*C_1_3_2_2/3),                                         C_1_1_3_3 + d\theta*(-2*sqrt(3)*C_1_1_1_3/3 + 2*sqrt(3)*C_1_1_2_3/3 - 2*sqrt(3)*C_1_2_3_3/3 + 2*sqrt(3)*C_1_3_3_3/3),                                          C_1_1_1_2 + d\theta*(sqrt(3)*C_1_1_1_1/3 - sqrt(3)*C_1_1_1_3/3 - sqrt(3)*C_1_1_2_2/3 + sqrt(3)*C_1_1_2_3/3 - 2*sqrt(3)*C_1_2_1_2/3 + 2*sqrt(3)*C_1_2_1_3/3),                                         C_1_1_1_3 + d\theta*(-sqrt(3)*C_1_1_1_1/3 + sqrt(3)*C_1_1_1_2/3 - sqrt(3)*C_1_1_2_3/3 + sqrt(3)*C_1_1_3_3/3 - 2*sqrt(3)*C_1_2_1_3/3 + 2*sqrt(3)*C_1_3_1_3/3),                                         C_1_1_2_3 + d\theta*(-sqrt(3)*C_1_1_1_2/3 + sqrt(3)*C_1_1_1_3/3 + sqrt(3)*C_1_1_

A material is isotropic with respect to a transformation of the basis when the elasticity tensor remains unchanged under that transformation:

\begin{equation*}
    \mathcal{C' _{ijkl}} = \mathcal{C _{ijkl}} \quad \text{for $M$ defined}
\end{equation*}

In [10]:
# Elasticity tensor for isotropic materials
def isotropy(M, type = 'P', *args):
    eq = []
    
    C = transformer(smp.eye(3))
    C_prime = transformer(M, type, *args)

    diff = C_prime - C
    for coefficient in diff:
        if type == 'R':
            coefficient = coefficient / dtheta
            
        equation = smp.Eq(coefficient, 0)
        eq.append(equation)

    sol = smp.solve(eq)
    return C.subs(sol)

In [11]:
isotropy(Rx, 'R', dtheta)

Matrix([
[C_1_1_1_1,                C_1_1_3_3,                C_1_1_3_3,         0,         0,         0],
[C_1_1_3_3,                C_3_3_3_3, -2*C_2_3_2_3 + C_3_3_3_3,         0,         0,         0],
[C_1_1_3_3, -2*C_2_3_2_3 + C_3_3_3_3,                C_3_3_3_3,         0,         0,         0],
[        0,                        0,                        0, C_1_3_1_3,         0,         0],
[        0,                        0,                        0,         0, C_1_3_1_3,         0],
[        0,                        0,                        0,         0,         0, C_2_3_2_3]])

In [12]:
isotropy(Ry, 'R', dtheta)

Matrix([
[               C_3_3_3_3, C_2_2_3_3, -2*C_1_3_1_3 + C_3_3_3_3,         0,         0,         0],
[               C_2_2_3_3, C_2_2_2_2,                C_2_2_3_3,         0,         0,         0],
[-2*C_1_3_1_3 + C_3_3_3_3, C_2_2_3_3,                C_3_3_3_3,         0,         0,         0],
[                       0,         0,                        0, C_2_3_2_3,         0,         0],
[                       0,         0,                        0,         0, C_1_3_1_3,         0],
[                       0,         0,                        0,         0,         0, C_2_3_2_3]])

In [13]:
isotropy(Rz, 'R', dtheta)

Matrix([
[               C_2_2_2_2, -2*C_1_2_1_2 + C_2_2_2_2, C_2_2_3_3,         0,         0,         0],
[-2*C_1_2_1_2 + C_2_2_2_2,                C_2_2_2_2, C_2_2_3_3,         0,         0,         0],
[               C_2_2_3_3,                C_2_2_3_3, C_3_3_3_3,         0,         0,         0],
[                       0,                        0,         0, C_1_2_1_2,         0,         0],
[                       0,                        0,         0,         0, C_2_3_2_3,         0],
[                       0,                        0,         0,         0,         0, C_2_3_2_3]])

In [14]:
isotropy(R111, 'R', dtheta)

Matrix([
[                                                     C_3_3_3_3,                                                      C_2_2_3_3,                                                      C_2_2_3_3,                                                      C_2_3_3_3,                                                      C_2_3_3_3, -2*C_1_3_2_3 + C_2_2_3_3 + 2*C_2_3_2_3 + C_2_3_3_3 - C_3_3_3_3],
[                                                     C_2_2_3_3,                                                      C_3_3_3_3,                                                      C_2_2_3_3,                                                      C_2_3_3_3, -2*C_1_3_2_3 + C_2_2_3_3 + 2*C_2_3_2_3 + C_2_3_3_3 - C_3_3_3_3,                                                      C_2_3_3_3],
[                                                     C_2_2_3_3,                                                      C_2_2_3_3,                                                      C_3_3_3_3, -2*C_1_3_2_3 + C_2_2_3_3 +

Let's see now some elementary basis changes

In [15]:
# Symmetries with respect to planes
Mxy = smp.Matrix([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, - 1]
])

Mxz = smp.Matrix([
    [1, 0, 0],
    [0, - 1, 0],
    [0, 0, 1]
])

Myz = smp.Matrix([
    [- 1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

Mxyz = smp.Matrix([
    [- 1, 0, 0],
    [0, - 1, 0],
    [0, 0, - 1]
])

In [16]:
transformer(Mxy)

Matrix([
[ C_1_1_1_1,  C_1_1_2_2,  C_1_1_3_3,  C_1_1_1_2, -C_1_1_1_3, -C_1_1_2_3],
[ C_1_1_2_2,  C_2_2_2_2,  C_2_2_3_3,  C_1_2_2_2, -C_1_3_2_2, -C_2_2_2_3],
[ C_1_1_3_3,  C_2_2_3_3,  C_3_3_3_3,  C_1_2_3_3, -C_1_3_3_3, -C_2_3_3_3],
[ C_1_1_1_2,  C_1_2_2_2,  C_1_2_3_3,  C_1_2_1_2, -C_1_2_1_3, -C_1_2_2_3],
[-C_1_1_1_3, -C_1_3_2_2, -C_1_3_3_3, -C_1_2_1_3,  C_1_3_1_3,  C_1_3_2_3],
[-C_1_1_2_3, -C_2_2_2_3, -C_2_3_3_3, -C_1_2_2_3,  C_1_3_2_3,  C_2_3_2_3]])

In [17]:
isotropy(Mxy)

Matrix([
[C_1_1_1_1, C_1_1_2_2, C_1_1_3_3, C_1_1_1_2,         0,         0],
[C_1_1_2_2, C_2_2_2_2, C_2_2_3_3, C_1_2_2_2,         0,         0],
[C_1_1_3_3, C_2_2_3_3, C_3_3_3_3, C_1_2_3_3,         0,         0],
[C_1_1_1_2, C_1_2_2_2, C_1_2_3_3, C_1_2_1_2,         0,         0],
[        0,         0,         0,         0, C_1_3_1_3, C_1_3_2_3],
[        0,         0,         0,         0, C_1_3_2_3, C_2_3_2_3]])

In [18]:
transformer(Mxz)

Matrix([
[ C_1_1_1_1,  C_1_1_2_2,  C_1_1_3_3, -C_1_1_1_2,  C_1_1_1_3, -C_1_1_2_3],
[ C_1_1_2_2,  C_2_2_2_2,  C_2_2_3_3, -C_1_2_2_2,  C_1_3_2_2, -C_2_2_2_3],
[ C_1_1_3_3,  C_2_2_3_3,  C_3_3_3_3, -C_1_2_3_3,  C_1_3_3_3, -C_2_3_3_3],
[-C_1_1_1_2, -C_1_2_2_2, -C_1_2_3_3,  C_1_2_1_2, -C_1_2_1_3,  C_1_2_2_3],
[ C_1_1_1_3,  C_1_3_2_2,  C_1_3_3_3, -C_1_2_1_3,  C_1_3_1_3, -C_1_3_2_3],
[-C_1_1_2_3, -C_2_2_2_3, -C_2_3_3_3,  C_1_2_2_3, -C_1_3_2_3,  C_2_3_2_3]])

In [19]:
isotropy(Mxz)

Matrix([
[C_1_1_1_1, C_1_1_2_2, C_1_1_3_3,         0, C_1_1_1_3,         0],
[C_1_1_2_2, C_2_2_2_2, C_2_2_3_3,         0, C_1_3_2_2,         0],
[C_1_1_3_3, C_2_2_3_3, C_3_3_3_3,         0, C_1_3_3_3,         0],
[        0,         0,         0, C_1_2_1_2,         0, C_1_2_2_3],
[C_1_1_1_3, C_1_3_2_2, C_1_3_3_3,         0, C_1_3_1_3,         0],
[        0,         0,         0, C_1_2_2_3,         0, C_2_3_2_3]])

In [20]:
transformer(Myz)

Matrix([
[ C_1_1_1_1,  C_1_1_2_2,  C_1_1_3_3, -C_1_1_1_2, -C_1_1_1_3,  C_1_1_2_3],
[ C_1_1_2_2,  C_2_2_2_2,  C_2_2_3_3, -C_1_2_2_2, -C_1_3_2_2,  C_2_2_2_3],
[ C_1_1_3_3,  C_2_2_3_3,  C_3_3_3_3, -C_1_2_3_3, -C_1_3_3_3,  C_2_3_3_3],
[-C_1_1_1_2, -C_1_2_2_2, -C_1_2_3_3,  C_1_2_1_2,  C_1_2_1_3, -C_1_2_2_3],
[-C_1_1_1_3, -C_1_3_2_2, -C_1_3_3_3,  C_1_2_1_3,  C_1_3_1_3, -C_1_3_2_3],
[ C_1_1_2_3,  C_2_2_2_3,  C_2_3_3_3, -C_1_2_2_3, -C_1_3_2_3,  C_2_3_2_3]])

In [21]:
isotropy(Myz)

Matrix([
[C_1_1_1_1, C_1_1_2_2, C_1_1_3_3,         0,         0, C_1_1_2_3],
[C_1_1_2_2, C_2_2_2_2, C_2_2_3_3,         0,         0, C_2_2_2_3],
[C_1_1_3_3, C_2_2_3_3, C_3_3_3_3,         0,         0, C_2_3_3_3],
[        0,         0,         0, C_1_2_1_2, C_1_2_1_3,         0],
[        0,         0,         0, C_1_2_1_3, C_1_3_1_3,         0],
[C_1_1_2_3, C_2_2_2_3, C_2_3_3_3,         0,         0, C_2_3_2_3]])

In [22]:
transformer(Mxyz)

Matrix([
[C_1_1_1_1, C_1_1_2_2, C_1_1_3_3, C_1_1_1_2, C_1_1_1_3, C_1_1_2_3],
[C_1_1_2_2, C_2_2_2_2, C_2_2_3_3, C_1_2_2_2, C_1_3_2_2, C_2_2_2_3],
[C_1_1_3_3, C_2_2_3_3, C_3_3_3_3, C_1_2_3_3, C_1_3_3_3, C_2_3_3_3],
[C_1_1_1_2, C_1_2_2_2, C_1_2_3_3, C_1_2_1_2, C_1_2_1_3, C_1_2_2_3],
[C_1_1_1_3, C_1_3_2_2, C_1_3_3_3, C_1_2_1_3, C_1_3_1_3, C_1_3_2_3],
[C_1_1_2_3, C_2_2_2_3, C_2_3_3_3, C_1_2_2_3, C_1_3_2_3, C_2_3_2_3]])